# First Experiment

---

In this notebook, run on [Colab](https://colab.research.google.com), we are going to perform our first experiment with **Detectron2** with our custom use case.

This notebook will be the first of the fourth experiments we have planned to perform, basicaly to check the impact of various initial learning rates within the training process of the model we have decided to mobilize for this study.

The chosen model is a Faster/Mask R-CNN COCO instance segmentation model, with ResNet/FPN backbone combination, based on ResNet-50 model and with a learning rate scheduler of 3 (more details can be found [here](https://github.com/facebookresearch/detectron2/blob/master/MODEL_ZOO.md)).

As detailed in the presentation page, we choose this one because it obtains the best speed/accuracy tradeoff.

Let's going on!

---

## Detectron2 installation and basic setup

In this section, we are going to adapt the procedure given in the notebook present in [Detectron2](https://github.com/facebookresearch/detectron2) GitHub homepage.

As always, the prerequisite step consists on loading the appropriate packages to perform our work:

In [1]:
# Needed packages:
import json, cv2, random, yaml
import pandas as pd
import numpy as np
from PIL import Image
from utils2 import rle_to_mask
#from google.colab.patches import cv2_imshow

Now, we need to check and get some configuration characteristics:

In [3]:
# Check what GPU is at our disposal in this session:
!nvidia-smi

Wed Mar 22 12:44:07 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.05    Driver Version: 525.85.05    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA TITAN Xp     Off  | 00000000:01:00.0 Off |                  N/A |
| 23%   34C    P8     9W / 250W |      6MiB / 12288MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

We can now begin the core installation process:

All is OK! We can now move on basic setup of the tool:

---

## Register the datasets

It's now time to register the datasets we have constituted in a previous notebook. As they are custom ones, we need to operate carefully and to follow the instructions given in the [Detectron2 custom dataset tutorial](https://detectron2.readthedocs.io/tutorials/datasets.html).

In [10]:
# Custom function to register custom dataset:
def get_severstal_dicts(dataset_path):
    """ Get Severstal dictionaries """
    
    # Open CSV description file:
    dataset_file_path = dataset_path + 'annotations.csv'
    dataset = pd.read_csv(dataset_file_path)
    
    # Initialize list of dictionaries for dataset:
    dataset_dicts = []
    
    # Go through dataset:
    for idx, row in dataset.iterrows():
        # Initialize record:
        record = {}
        # Populate first elements of record:
        file_name = dataset_path + row['ImageId']
        width, height = Image.open(file_name).size
        record['file_name'] = file_name
        record['height'] = height
        record['width'] = width
        record['image_id'] = idx
        # Initialize last element of record:
        annotations = []
        # Get mask:
        rle = row['EncodedPixels']
        mask = rle_to_mask(rle)
        # Determine contours:
        contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        # Populate last element of record:
        for contour in contours:
            contour_array = contour.reshape(-1, 2)
            px = [i[0] for i in contour_array]
            py = [j[1] for j in contour_array]
            poly = [(x + 0.5, y + 0.5) for x, y in zip(px, py)]
            poly = [p for x in poly for p in x]
            x0, y0 = int(np.min(px)), int(np.min(py))
            x1, y1 = int(np.max(px)), int(np.max(py))
            if (len(poly)%2)==0 and len(poly)>=6:
                annotation = {'bbox': [x0, y0, x1, y1],
                              
                              'segmentation': [poly],
                              'category_id': row['ClassId'] - 1,
                              'iscrowd': 0}
                annotations.append(annotation)
        record['annotations'] = annotations
        # Add to list of dictionaries:
        dataset_dicts.append(record)
    
    # Return results:
    return dataset_dicts

In [5]:
# Set annotations:
severstal_annotations = ['Type 1',
                         'Type 2',
                         'Type 3',
                         'Type 4']

In [11]:
# Register training and testing datasets:
for ds in ['training_ds', 'testing_ds']:
    DatasetCatalog.register(ds, lambda ds=ds: get_severstal_dicts('../data/' + ds + '/'))
    MetadataCatalog.get(ds).set(thing_classes=severstal_annotations)

NameError: name 'DatasetCatalog' is not defined

In [11]:
# Set metadata:
severstal_metadata = MetadataCatalog.get('training_ds')

In [13]:
# Get training and testing datasets dictionaries lists:
training_ds_dicts = get_severstal_dicts('data/severstal-steel-defect-detection/train_ds/')
testing_ds_dicts = get_severstal_dicts('data/severstal-steel-defect-detection/val_ds/')

In [14]:
# Display random recording:
print('*** A random recording from testing dataset:\n')
print(testing_ds_dicts[2092])

*** A random recording from testing dataset:

{'file_name': 'data/severstal-steel-defect-detection/val_ds/fea8e46ad.jpg', 'height': 256, 'width': 1600, 'image_id': 2092, 'annotations': [{'bbox': [1339, 24, 1353, 255], 'segmentation': [[1347.5, 24.5, 1346.5, 25.5, 1344.5, 25.5, 1343.5, 26.5, 1341.5, 26.5, 1340.5, 27.5, 1339.5, 27.5, 1339.5, 141.5, 1340.5, 142.5, 1340.5, 255.5, 1353.5, 255.5, 1353.5, 195.5, 1352.5, 194.5, 1352.5, 140.5, 1351.5, 139.5, 1351.5, 107.5, 1350.5, 106.5, 1350.5, 74.5, 1349.5, 73.5, 1349.5, 41.5, 1348.5, 40.5, 1348.5, 24.5]], 'category_id': 2, 'iscrowd': 0}, {'bbox': [1321, 21, 1336, 254], 'segmentation': [[1321.5, 21.5, 1321.5, 131.5, 1322.5, 132.5, 1322.5, 254.5, 1336.5, 254.5, 1336.5, 226.5, 1335.5, 225.5, 1335.5, 168.5, 1334.5, 167.5, 1334.5, 109.5, 1333.5, 108.5, 1333.5, 51.5, 1332.5, 50.5, 1332.5, 21.5]], 'category_id': 2, 'iscrowd': 0}, {'bbox': [710, 2, 735, 255], 'segmentation': [[716.5, 2.5, 716.5, 9.5, 715.5, 10.5, 715.5, 27.5, 714.5, 28.5, 714.5, 45.

In [16]:
# Display random recording:
print('*** A random recording from testing dataset:\n')
print(training_ds_dicts[2])

*** A random recording from testing dataset:

{'file_name': 'data/severstal-steel-defect-detection/train_ds/000f6bf48.jpg', 'height': 256, 'width': 1600, 'image_id': 2, 'annotations': [{'bbox': [853, 126, 1131, 255], 'segmentation': [[978.5, 126.5, 977.5, 127.5, 977.5, 128.5, 976.5, 129.5, 976.5, 130.5, 973.5, 133.5, 973.5, 134.5, 968.5, 139.5, 968.5, 140.5, 964.5, 144.5, 964.5, 145.5, 956.5, 153.5, 956.5, 154.5, 945.5, 165.5, 945.5, 166.5, 940.5, 171.5, 938.5, 171.5, 937.5, 172.5, 933.5, 172.5, 932.5, 173.5, 929.5, 173.5, 928.5, 174.5, 924.5, 174.5, 923.5, 175.5, 916.5, 175.5, 915.5, 174.5, 909.5, 174.5, 908.5, 173.5, 905.5, 173.5, 904.5, 172.5, 902.5, 172.5, 901.5, 171.5, 898.5, 171.5, 897.5, 170.5, 895.5, 170.5, 894.5, 169.5, 892.5, 169.5, 891.5, 168.5, 888.5, 168.5, 887.5, 167.5, 885.5, 167.5, 884.5, 166.5, 881.5, 166.5, 880.5, 165.5, 879.5, 165.5, 878.5, 166.5, 877.5, 166.5, 876.5, 167.5, 875.5, 167.5, 874.5, 168.5, 873.5, 168.5, 872.5, 169.5, 871.5, 169.5, 870.5, 170.5, 869.5, 17

To verify the data loading is correct, let's visualize the annotations of randomly selected samples in the testing set:

In [17]:
type(training_ds_dicts)

list

In [15]:
# Some random visualizations:
for recording in random.sample(testing_ds_dicts, 3):
    img = cv2.imread(recording['file_name'])
    visualizer = Visualizer(img[:, :, ::-1],
                            metadata=severstal_metadata,
                            scale=0.5)
    out = visualizer.draw_dataset_dict(recording)
    cv2_imshow(out.get_image()[:, :, ::-1])

NameError: name 'Visualizer' is not defined

In [20]:
import json
j_obj = json.dumps(training_ds_dicts)

# Writing to sample.json
with open("training_ds.json", "w") as outfile:
    outfile.write(j_obj)


In [21]:
import json
j_obj = json.dumps(testing_ds_dicts)

# Writing to sample.json
with open("testing_ds.json", "w") as outfile:
    outfile.write(j_obj)